In [3]:
import requests
import pandas as pd
import pprint

In [4]:
def get_stock_history(date, stock_no, retry = 5):   #從www.twse.com.tw讀取資料
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, stock_no)
    r = requests.get(url)
    data = r.json()
    print(data)
    return data
    #return transform(data['data'])  #進行資料格式轉換

In [5]:
a = get_stock_history('20180101', '2330')
pprint.pprint(a)

{'stat': 'OK', 'date': '20180101', 'title': '107年01月 2330 台積電           各日成交資訊', 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'], 'data': [['107/01/02', '18,055,269', '4,188,555,408', '231.50', '232.50', '231.00', '232.50', '+3.00', '9,954'], ['107/01/03', '31,706,091', '7,504,382,512', '236.00', '238.00', '235.50', '237.00', '+4.50', '13,633'], ['107/01/04', '29,179,613', '6,963,192,636', '240.00', '240.00', '236.50', '239.50', '+2.50', '10,953'], ['107/01/05', '23,721,255', '5,681,934,695', '240.00', '240.00', '238.00', '240.00', '+0.50', '8,659'], ['107/01/08', '21,846,692', '5,281,823,362', '242.00', '242.50', '240.50', '242.00', '+2.00', '10,251'], ['107/01/09', '19,043,123', '4,588,314,012', '242.00', '242.00', '239.50', '242.00', ' 0.00', '7,124'], ['107/01/10', '25,716,220', '6,118,683,273', '241.50', '242.00', '236.00', '236.50', '-5.50', '10,534'], ['107/01/11', '32,070,338', '7,500,674,455', '235.00', '236.00', '232.50', '235.00', '-1.50', '9,199

In [32]:
x= pandas.DataFrame({ e[0]:e[1:] for e in a['data']})
#print(x)
x.index =a['fields'][1:]
#print (x);
x= x.transpose()
print(x)

                 成交股數            成交金額     開盤價     最高價     最低價     收盤價   漲跌價差  \
107/01/02  18,055,269   4,188,555,408  231.50  232.50  231.00  232.50  +3.00   
107/01/03  31,706,091   7,504,382,512  236.00  238.00  235.50  237.00  +4.50   
107/01/04  29,179,613   6,963,192,636  240.00  240.00  236.50  239.50  +2.50   
107/01/05  23,721,255   5,681,934,695  240.00  240.00  238.00  240.00  +0.50   
107/01/08  21,846,692   5,281,823,362  242.00  242.50  240.50  242.00  +2.00   
107/01/09  19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   0.00   
107/01/10  25,716,220   6,118,683,273  241.50  242.00  236.00  236.50  -5.50   
107/01/11  32,070,338   7,500,674,455  235.00  236.00  232.50  235.00  -1.50   
107/01/12  23,141,291   5,463,302,207  234.50  238.00  233.50  237.00  +2.00   
107/01/15  28,576,533   6,832,851,887  240.00  240.00  238.00  240.00  +3.00   
107/01/16  23,407,632   5,609,009,540  240.00  240.50  238.00  240.50  +0.50   
107/01/17  38,118,119   9,207,582,787  2

In [60]:
def get_stock_price_month(date, stock_id):
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, str(stock_id))
    r=requests.get(url)
    data = r.json()
    df = pd.DataFrame({e[0]: e[1:]  for e in data['data']})
    df.index=data['fields'][1:]
    #df =df.transpose
    def taiwan_year_to_year(tw_year):
        dates = tw_year.split('/')
        dates[0] = str(int(dates[0])+1911)
        return '/'.join(dates)
    
    df.columns = df.columns.map(taiwan_year_to_year)
    df.columns = pd.to_datetime(df.columns,format='%Y/%m/%d')
    df = df.transpose()
    print(df)
    return df
    
    


In [20]:

import datetime 
from datetime import timedelta
import calendar

time_format_api = "%Y/%m/%d"
time_format_string = "%Y%m%d"

def taiwan_year_to_year(tw_year):
        dates = tw_year.split('/')
        dates[0] = str(int(dates[0])+1911)
        return '/'.join(dates)
    
def string_to_datetime(string_date,request=False):
    datetime_format = time_format_api if request else time_format_string
    return datetime.datetime.strptime(string_date,datetime_format )

def datetime_to_string(date):
    return date.strftime(time_format_string)

def get_stock_price_monthly(stock_id,date,end=None):
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, str(stock_id))
    r=requests.get(url)
    data = r.json()
    def process_datetime(all_data):
        all_data[0] = string_to_datetime(taiwan_year_to_year(all_data[0]),True)
        return all_data
    
    data['data']= list(map(process_datetime,data['data']))

    if end:
        dt_end = string_to_datetime(end)
        data['data']=list(filter(lambda in_data:  in_data[0]<= dt_end,data['data']))
    return data
        

def add_months(sourcedate,months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day,calendar.monthrange(year,month)[1])
    return datetime.datetime(year,month,day,0,0)
    
def get_stock_price(stock_id, from_date, to_date=None):
    date_from =string_to_datetime(from_date)
    if to_date:
        date_to = string_to_datetime(to_date)
    else:
        date_to = datetime.datetime.now()
    date_check = date_from
    prices= {'data':[]}
    while date_check<= date_to:
        price_data = get_stock_price_monthly(stock_id, datetime_to_string(date_check),datetime_to_string(date_to))
        prices['data'].extend(price_data['data'])
        if 'fields' not in prices:
            prices['fields'] = price_data['fields']
        date_check = add_months(date_check,1)
    return prices
        
        
#get_stock_price_monthly(2330,'20180101','20180105')
data = get_stock_price(2330,'20180101','20180212')
pprint.pprint(data)



{'data': [[datetime.datetime(2018, 1, 2, 0, 0),
           '18,055,269',
           '4,188,555,408',
           '231.50',
           '232.50',
           '231.00',
           '232.50',
           '+3.00',
           '9,954'],
          [datetime.datetime(2018, 1, 3, 0, 0),
           '31,706,091',
           '7,504,382,512',
           '236.00',
           '238.00',
           '235.50',
           '237.00',
           '+4.50',
           '13,633'],
          [datetime.datetime(2018, 1, 4, 0, 0),
           '29,179,613',
           '6,963,192,636',
           '240.00',
           '240.00',
           '236.50',
           '239.50',
           '+2.50',
           '10,953'],
          [datetime.datetime(2018, 1, 5, 0, 0),
           '23,721,255',
           '5,681,934,695',
           '240.00',
           '240.00',
           '238.00',
           '240.00',
           '+0.50',
           '8,659'],
          [datetime.datetime(2018, 1, 8, 0, 0),
           '21,846,692',
           '5,281,823

In [29]:
def get_price_report(stock_id, from_date, to_date=None):
    prices_dict = get_stock_price(stock_id, from_date,to_date)
    df = pd.DataFrame({ e[0]: e[1:] for e in prices_dict['data']}, index=prices_dict['fields'][1:])
    df = df.transpose()
    #print(df)
    return df
    
get_price_report(2330,'20180101','20180312')

成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
2018-01-02   18,055,269   4,188,555,408  231.50  232.50  231.00  232.50   
2018-01-03   31,706,091   7,504,382,512  236.00  238.00  235.50  237.00   
2018-01-04   29,179,613   6,963,192,636  240.00  240.00  236.50  239.50   
2018-01-05   23,721,255   5,681,934,695  240.00  240.00  238.00  240.00   
2018-01-08   21,846,692   5,281,823,362  242.00  242.50  240.50  242.00   
2018-01-09   19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   
2018-01-10   25,716,220   6,118,683,273  241.50  242.00  236.00  236.50   
2018-01-11   32,070,338   7,500,674,455  235.00  236.00  232.50  235.00   
2018-01-12   23,141,291   5,463,302,207  234.50  238.00  233.50  237.00   
2018-01-15   28,576,533   6,832,851,887  240.00  240.00  238.00  240.00   
2018-01-16   23,407,632   5,609,009,540  240.00  240.50  238.00  240.50   
2018-01-17   38,118,119   9,207,582,787  240.50  243.00  239.00  242.00   
2018-01-18   50,119,952  12,406,562,364  245.00  250.00  245.00  248.50   
2018-01-19   55,061,292  13,975,174,348  253.50  255.50  251.50  255.50   
2018-01-22   45,907,509  11,934,882,643  257.50  262.00  257.00  261.50   
2018-01-23   34,606,444   9,155,080,569  262.50  266.00  262.50  266.00   
2018-01-24   42,600,813  11,022,372,004  263.00  263.00  256.50  258.00   
2018-01-25   46,214,756  11,981,089,514  258.00  264.00  256.50  258.00   
2018-01-26   43,514,523  11,101,977,348  256.50  257.50  253.50  255.00   
2018-01-29   31,306,234   8,067,780,117  259.00  261.50  255.00  258.50   
2018-01-30   37,410,591   9,523,980,994  256.00  257.50  252.50  253.00   
2018-01-31   45,807,808  11,670,582,000  253.00  257.50  252.00  255.00   
2018-02-01   30,900,457   8,022,694,430  257.50  261.00  257.00  259.50   
2018-02-02   25,707,560   6,633,344,540  259.00  260.00  255.00  259.50   
2018-02-05   46,554,379  11,742,299,859  251.00  254.50  250.50  253.00   
2018-02-06  104,448,048  24,944,565,955  242.50  243.50  234.00  239.00   
2018-02-07   82,299,178  19,903,032,966  245.00  245.50  239.50  240.00   
2018-02-08   31,192,550   7,468,970,072  239.50  242.00  238.00  238.50   
2018-02-09   65,939,265  15,227,192,540  230.00  233.00  228.50  232.50   
2018-02-12   42,917,236  10,113,012,768  235.50  237.50  235.00  236.50   
2018-02-21   76,396,174  18,582,475,956  244.50  244.50  242.00  242.50   
2018-02-22   45,492,993  10,977,665,819  244.00  244.00  239.00  239.50   
2018-02-23   26,985,776   6,578,657,620  241.00  245.00  241.00  245.00   
2018-02-26   36,165,810   8,910,352,170  247.00  247.50  245.00  246.50   
2018-02-27   44,699,119  11,061,945,363  250.50  250.50  246.00  246.00   
2018-03-01   43,847,984  10,669,194,561  244.00  245.00  242.00  243.00   
2018-03-02   35,289,170   8,453,712,014  240.00  241.00  238.50  240.00   
2018-03-05   27,337,846   6,607,367,732  242.50  243.00  240.50  241.50   
2018-03-06   36,945,780   9,155,812,210  245.50  250.00  245.50  250.00   
2018-03-07   30,391,219   7,525,601,638  248.00  248.50  246.00  247.00   
2018-03-08   20,645,694   5,172,732,531  249.50  251.50  249.00  249.50   
2018-03-09   22,887,063   5,723,277,170  250.00  251.00  248.50  250.50   
2018-03-12   25,100,615   6,370,529,728  252.00  255.00  251.50  254.00   

              漲跌價差    成交筆數  
2018-01-02   +3.00   9,954  
2018-01-03   +4.50  13,633  
2018-01-04   +2.50  10,953  
2018-01-05   +0.50   8,659  
2018-01-08   +2.00  10,251  
2018-01-09    0.00   7,124  
2018-01-10   -5.50  10,534  
2018-01-11   -1.50   9,199  
2018-01-12   +2.00   7,905  
2018-01-15   +3.00   9,756  
2018-01-16   +0.50   8,156  
2018-01-17   +1.50  12,593  
2018-01-18   +6.50  19,482  
2018-01-19   +7.00  18,801  
2018-01-22   +6.00  13,558  
2018-01-23   +4.50  13,993  
2018-01-24   -8.00  17,287  
2018-01-25    0.00  15,826  
2018-01-26   -3.00  12,821  
2018-01-29   +3.50  12,211  
2018-01-30   -5.50  12,987  
2018-01-31   +2.00  13,553  
2018-02-01   +4.50  13,598  
2018-02-02    0.00  11

In [62]:
dt = get_stock_price_month('20180101',2330)
dt.describe()

                  成交股數            成交金額     開盤價     最高價     最低價     收盤價   漲跌價差  \
2018-01-02  18,055,269   4,188,555,408  231.50  232.50  231.00  232.50  +3.00   
2018-01-03  31,706,091   7,504,382,512  236.00  238.00  235.50  237.00  +4.50   
2018-01-04  29,179,613   6,963,192,636  240.00  240.00  236.50  239.50  +2.50   
2018-01-05  23,721,255   5,681,934,695  240.00  240.00  238.00  240.00  +0.50   
2018-01-08  21,846,692   5,281,823,362  242.00  242.50  240.50  242.00  +2.00   
2018-01-09  19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   0.00   
2018-01-10  25,716,220   6,118,683,273  241.50  242.00  236.00  236.50  -5.50   
2018-01-11  32,070,338   7,500,674,455  235.00  236.00  232.50  235.00  -1.50   
2018-01-12  23,141,291   5,463,302,207  234.50  238.00  233.50  237.00  +2.00   
2018-01-15  28,576,533   6,832,851,887  240.00  240.00  238.00  240.00  +3.00   
2018-01-16  23,407,632   5,609,009,540  240.00  240.50  238.00  240.50  +0.50   
2018-01-17  38,118,119   9,2

成交股數           成交金額     開盤價     最高價     最低價     收盤價   漲跌價差  \
count           22             22      22      22      22      22     22   
unique          22             22      18      16      19      16     15   
top     23,141,291  9,155,080,569  240.00  257.50  238.00  242.00  +2.00   
freq             1              1       4       3       3       3      3   

          成交筆數  
count       22  
unique      22  
top     15,826  
freq         1

In [40]:
import get_stock_price as twse

def datetime_to_google_time(time):
    month_abbr = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
                  'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
                  'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    #year=time.strftime("%Y")
    return time.strftime("%d-%b-%y")

def twse_to_google(twse_dataframe, to_csv_filename=None):
    google_dataframe = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume'])
    #print(twse_dataframe.index.values)
    google_dataframe['Date']= twse_dataframe.index.values
    #twse_dataframe.index.apply(lambda t:t.strftime("%d-%b-%y"))
    google_dataframe['Open']=twse_dataframe['開盤價'].values
    google_dataframe['High']=twse_dataframe['最高價'].values
    google_dataframe['Low']=twse_dataframe['最低價'].values
    google_dataframe['Close']=twse_dataframe['收盤價'].values
    google_dataframe['Volume'] = twse_dataframe['成交股數'].values
    google_dataframe['Date']= google_dataframe['Date'].apply(lambda t:t.strftime("%d-%b-%y"))
    if to_csv_filename:
        google_dataframe.to_csv(to_csv_filename,index=False)
    return google_dataframe
        
gdf = twse_to_google(get_price_report(2330,'20180101','20180312'))
print(gdf)

         Date    Open    High     Low   Close       Volume
0   02-Jan-18  231.50  232.50  231.00  232.50   18,055,269
1   03-Jan-18  236.00  238.00  235.50  237.00   31,706,091
2   04-Jan-18  240.00  240.00  236.50  239.50   29,179,613
3   05-Jan-18  240.00  240.00  238.00  240.00   23,721,255
4   08-Jan-18  242.00  242.50  240.50  242.00   21,846,692
5   09-Jan-18  242.00  242.00  239.50  242.00   19,043,123
6   10-Jan-18  241.50  242.00  236.00  236.50   25,716,220
7   11-Jan-18  235.00  236.00  232.50  235.00   32,070,338
8   12-Jan-18  234.50  238.00  233.50  237.00   23,141,291
9   15-Jan-18  240.00  240.00  238.00  240.00   28,576,533
10  16-Jan-18  240.00  240.50  238.00  240.50   23,407,632
11  17-Jan-18  240.50  243.00  239.00  242.00   38,118,119
12  18-Jan-18  245.00  250.00  245.00  248.50   50,119,952
13  19-Jan-18  253.50  255.50  251.50  255.50   55,061,292
14  22-Jan-18  257.50  262.00  257.00  261.50   45,907,509
15  23-Jan-18  262.50  266.00  262.50  266.00   34,606,4